In [45]:
import pandas as pd
import numpy as np
import requests
from get_mail_mailru import main as mail_main
from parser_broker_report import main as parser_main

In [46]:
# mail_main()
# parser_main()

In [47]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Дата'] = portfolio['Дата'].astype('datetime64[ns]')
transactions = pd.read_csv('transactions.csv').drop_duplicates(['Дата заключения', 'Время заключения', 'Статус'])
# transactions['Дата заключения'] = pd.to_datetime(transactions['Дата заключения'] + ' ' + transactions['Время заключения'], dayfirst=True)
# transactions['Дата расчетов'] = pd.to_datetime(transactions['Дата расчетов'], dayfirst=True)
transactions = transactions.drop('Время заключения', axis=1).dropna(axis=1).sort_values('Дата заключения').reset_index(drop=True)
transactions_executed = transactions[transactions['Статус'] == 'И']

In [48]:
ticker_list = list(transactions_executed['Код'].unique())
share_split_dict = {
    'FXGD': ['2022-02-17', 10],
    'SBMX': ['2021-06-09', 100],
    'FXUS': ['2022-01-24', 100],
    'FXRL': ['2021-11-24', 100],
    'FXRU': ['2022-02-17', 10],
    'FXDE': ['2021-12-15', 100]
}
replacement_dict = {
    'VTBE': 'RSHE',
    'RU000A102HB1': 'SU26227RMFS7'
}

In [49]:
def fix_split():
    ticker_list_copy = ticker_list.copy()
    for ticker in ticker_list_copy:
        if ticker == 'RU000A102HB1':
            new_ticker = 'SU26227RMFS7'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
        if ticker == 'VTBE':
            new_ticker = 'RSHE'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
    for ticker in ticker_list:
        if ticker in share_split_dict:
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] = \
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] * \
            share_split_dict[ticker][1]


fix_split()

In [50]:
def get_share_price_dict(tickers: list):
    share_price_dict = {}
    for ticker in tickers:
        if ticker in ['SBMX', 'AFKS']:
            round_numb = 3
        elif ticker in ['IRAO']:
            round_numb = 4
        else:
            round_numb = 2
        share = transactions_executed[transactions_executed['Код'] == ticker]
        share_price = ((share['Сумма'].sum() + share['Комиссия Брокера'].sum() + share['Комиссия Биржи'].sum()) / 
                            (share.loc[share['Вид'] == 'Покупка', 'Количество'].sum() - share.loc[share['Вид'] == 'Продажа', 'Количество'].sum())).round(round_numb)
        share_price_dict[ticker] = share_price
    return share_price_dict


share_price_dict = get_share_price_dict(ticker_list)

In [51]:
def get_share_amount_dict(tickers: list):
    share_amount_dict = {}
    for ticker in tickers:
        share_amount_dict[ticker] = transactions_executed.loc[transactions_executed['Код'] == ticker, 'Количество'].sum()
    return share_amount_dict


share_amount_dict = get_share_amount_dict(ticker_list)

In [52]:
def get_trading_dict():
    trading_dict = {}
    trading_mode_list = ['TQBR', 'TQTF', 'TQCB', 'TQOB']
    for id_trading in trading_mode_list:
        stocks = 'shares'
        if id_trading in ['TQOB', 'TQCB']:
            stocks = 'bonds'
        url = (f"https://iss.moex.com/iss/engines/stock/markets/{stocks}/boards/{id_trading}/"
            f"securities.csv?iss.meta=off&iss.only=marketdata&marketdata.columns=SECID,LAST")
        csv_text = requests.get(url).text.split('\n')
        trading_dict[id_trading] = csv_text
    return trading_dict


def get_last_prices_dict(tickers: list):
    last_prices_dict = {}
    trading_dict = get_trading_dict()
    for ticker in tickers:
        if ticker == 'RU000A102HB1':
            ticker =  'SU26227RMFS7'
        if ticker == 'VTBE':
            ticker = 'RSHE'
        for value in trading_dict.values():
            for line in value:
                line = line.split(';')
                if ticker in line:
                    last_prices_dict[ticker] = line[1]
    return last_prices_dict


last_prices_dict = get_last_prices_dict(ticker_list)

In [53]:
def get_commission_dict(tickers: list):
    commission_dict = {}
    for ticker in tickers:
        commission_dict[ticker] = (transactions_executed.loc[transactions_executed['Код'] == ticker, 'Комиссия Брокера'] + \
                                     transactions_executed.loc[transactions_executed['Код'] == ticker, 'Комиссия Биржи']) \
                                    .sum().round(2)
    return commission_dict


commission_dict = get_commission_dict(ticker_list)

In [54]:
def get_coupon_dict(tickers: list):
    coupon_dict = {}
    url = "https://iss.moex.com/iss/engines/stock/markets/bonds/boards/TQOB/securities.csv?iss.meta=off&iss.only" \
          "=securities&securities.columns=SECID,ACCRUEDINT "
    csv_text = requests.get(url).text.split('\n')

    for ticker in tickers:
        for line in csv_text:
            line = line.split(';')
            if ticker in line:
                coupon_dict[ticker] = line[1]
    return coupon_dict


coupon_dict = get_coupon_dict(ticker_list)

In [56]:
portfolio_dict = {
    'Котировки': last_prices_dict,
    'НКД': coupon_dict,
    'Количество': share_amount_dict,
    'Средняя цена': share_price_dict,
    'Комиссия': commission_dict
}

main = pd.DataFrame.from_dict(portfolio_dict)
main.index.name = 'Название'
main.loc[main['Котировки'] == '', 'Котировки'] = np.nan
main.dropna(axis=0, subset='Котировки', inplace=True)
main[['Котировки', 'НКД']] = main[['Котировки', 'НКД']].astype('float64')
# main.loc['SU25084RMFS3', 'Котировки'] = main.loc['SU25084RMFS3', 'Котировки'] * 10
# main.loc['SU25084RMFS3', 'Текущая цена'] = (main.loc['SU25084RMFS3', 'Котировки'] + main.loc['SU25084RMFS3', 'НКД']) * main.loc['SU25084RMFS3', 'Количество']
main['Текущая цена'] = main['Котировки'] * main['Количество']
main['P/L, руб.'] = main['Текущая цена'] - main['Средняя цена'] * main['Количество']
main['P/L, %'] = (main['Котировки'] * 100 / main['Средняя цена'] - 100).round(2)

# bond_price, bond_NKD, bond_amount, bond_average_price, bond_commission = \
#             main.loc['SU25084RMFS3', 'Котировки'], \
#             main.loc['SU25084RMFS3', 'НКД'], \
#             main.loc['SU25084RMFS3', 'Количество'], \
#             main.loc['SU25084RMFS3', 'Средняя цена'], \
#             main.loc['SU25084RMFS3', 'Комиссия']

# main.loc['SU25084RMFS3', 'Котировки'] = bond_price * 10
# main.loc['SU25084RMFS3', 'Текущая цена'] = (bond_price * 10 + bond_NKD) * bond_amount
# main.loc['SU25084RMFS3', 'P/L, руб.'] = main.loc['SU25084RMFS3', 'Текущая цена'] - bond_average_price * bond_amount
# main.loc['SU25084RMFS3', 'P/L, %'] = (((bond_price * 10 + bond_NKD) * 100 / bond_average_price) - 100).round(2)

main.to_csv('portfolio_main.csv', index_label=False)

In [57]:
df = pd.read_csv('portfolio_main.csv')
df.index.name = 'Название'

In [58]:
df

,Котировки,НКД,Количество,Средняя цена,Комиссия,Текущая цена,"P/L, руб.","P/L, %"
Название,,,,,,,,
SBER,274.720,NaN,50,233.9000,10.50,13736.000,2041.000,17.45
SBMX,17.812,NaN,27461,0.4760,4.98,489135.332,476063.896,3642.02
MGNT,6376.000,NaN,2,4852.1100,8.72,12752.000,3047.780,31.41
LKOH,7177.000,NaN,2,5995.3200,8.63,14354.000,2363.360,19.71
GAZP,161.860,NaN,40,173.2800,6.24,6474.400,-456.800,-6.59
YNDX,2414.400,NaN,1,2642.3800,2.38,2414.400,-227.980,-8.63
CHMF,1273.600,NaN,4,1292.5700,4.27,5094.400,-75.880,-1.47
AFKS,16.043,NaN,300,17.8960,4.84,4812.900,-555.900,-10.35
IRAO,4.036,NaN,1000,4.1869,3.52,4036.000,-150.900,-3.60
